In [1]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [3]:
model = tf.keras.models.load_model('pythonProject2/model-best')

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 66)               0         
 torization)                                                     
                                                                 
 token_embedding (Embedding)  (None, 66, 128)          6306560   
                                                                 
 conv1d (Conv1D)             (None, 66, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 output_layer (Dense)        (None, 3)                 195   

In [6]:
model.save('model-best, save_format='tf'')

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<keras.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x000001A8C8F28670>]

array(['Negative', 'Neutral', 'Positive']


In [5]:
from newsapi import NewsApiClient
from datetime import datetime, timedelta
from config import getAPI
(datetime.today() - timedelta(1)).strftime('%Y-%m-%d')

'2024-08-13'

In [56]:
# Initialize the client with your API key
api_key = getAPI()
newsapi = NewsApiClient(api_key=api_key)

# Get today's date in the format YYYY-MM-DD
yesterday = (datetime.today() - timedelta(10)).strftime('%Y-%m-%d')
today = datetime.today().strftime('%Y-%m-%d')

# Fetch articles related to Tesla from today
tesla_articles = newsapi.get_everything(
    q='figma',
    from_param=yesterday,
    to = today,
    language='en',
    sort_by='relevancy'
)

# Print the titles and descriptions of the articles
for article in tesla_articles['articles']:
    print(f"Title: {article['title']}")
    print(f"Description: {article['description']}")
    print(f"URL: {article['url']}\n")

Title: How we migrated onto K8s in less than 12 months
Description: Migrating onto Kubernetes can take years. Here’s why we decided it was worth undertaking, and how we moved a majority of our core services.
URL: https://www.figma.com/blog/migrating-onto-kubernetes/

Title: 10 best home office software tools for productivity and collaboration
Description: Boost your teamwork and crush goals with these home office heroes
URL: https://www.xda-developers.com/best-home-office-software-for-productivity-and-collaboration/

Title: Infisical (YC W23) Is Hiring a Customer Success Engineer
Description: About Infisical
Infisical (https://infisical.com) is the #1 open source secret management platform for developers. In other words, we help organizations manage API-keys, DB access tokens, certificates, and other credentials across all parts of their infra! In…
URL: https://www.ycombinator.com/companies/infisical/jobs/28DIh6L-customer-success-engineer

Title: Free AI-Website Builder, Scene, Helps W

In [33]:
list = []
for article in tesla_articles['articles']:
    list.append(article['title'] + ' ' + article['description'])
list

['Rivian is bringing Apple Music with spatial audio to its CarPlay-less EVs Rivian is pushing a new software update that includes Apple Music, but it requires a Connect Plus subscription to work on the go, while CarPlay is still not an option.',
 'Fiery Cybertruck Death Spurs Federal Probe A man was killed in a Cybertruck crash in Texas early Monday morning.',
 "Elon Musk initially said the Cybertruck would cost from $39,900. Now, you can't buy it for much less than $100,000. Tesla removed the option to preorder the cheaper versions of its Cybertruck from its website last week.",
 "Uber CEO says there's a potentially big problem with Elon Musk's Robotaxi idea Uber CEO Dara Khosrowshahi is skeptical that Tesla can pull off its Robotaxi plan. But he hopes the two companies can collaborate.",
 "Californians are buying fewer Teslas. That could be a bad sign for Elon Musk. Tesla's sliding sales in California could be a symptom of wider problems for Elon Musk and his EV maker, experts say.",

In [72]:
import re
from nltk.corpus import stopwords

def preprocess(text):
    
    text = text.lower()

    # remove punctuations and numbers 
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text) #when we remove apostrophe, is it replaced by a white space

    # remove all single characters and replace it by a space
    text = re.sub(r'\s+', ' ', text)

    # remove stopwords
    stop_words = stopwords.words('english')
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')

    text = pattern.sub('', text)
    return text
def processArticles(articles):
    list = []
    for article in articles['articles']:
        # get title and description, defaulting to an empty string if they're not strings
        title = article.get('title', '')
        description = article.get('description', '')
        #further make sure they are strings by replacing with '' if not string
        if not isinstance(title, str):
            title = ''
        if not isinstance(description, str):
            description = ''
        title_and_description = preprocess(title + ' ' + description)
        list.append(title_and_description)
    return list
def getArticles(input):
    # Initialize the client with your API key
    api_key = getAPI()
    newsapi = NewsApiClient(api_key=api_key)
    
    # Get today's date in the format YYYY-MM-DD
    yesterday = (datetime.today() - timedelta(10)).strftime('%Y-%m-%d')
    today = datetime.today().strftime('%Y-%m-%d')
    
    # Fetch articles related to Tesla from today
    articles = newsapi.get_everything(
        q=input,
        from_param=yesterday,
        to = today,
        language='en',
        sort_by='relevancy'
    )
    
    return processArticles(articles)


In [73]:
text_vectorization = TextVectorization(
    max_tokens=68000,
    output_sequence_length=int(78.0)
)
def run_model(inp):
    # Get the list of articles or sentences
    list = getArticles(inp)

    if not list: 
        return "Empty! No recent news available here."
    
    # Initialize a dictionary to keep track of sentiment counts
    sentiment_counts = {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    # Predict sentiment for each input sentence
    for input_sentence in list:
        predictions = model.predict([input_sentence])
        
        # Get the predicted class
        predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
        
        # Map the predicted class to the corresponding sentiment label
        sentiment_labels = ["Negative", "Neutral", "Positive"]
        predicted_sentiment = sentiment_labels[predicted_class]
        
        # Increment the count for the predicted sentiment
        sentiment_counts[predicted_sentiment] += 1
    
    return sentiment_counts

In [76]:
import gradio as gr

# gradio interface
iface = gr.Interface(
    fn=create_buttons,                # Function to process input
    inputs=['text'],
    outputs=["text"],                    # Output as text
    title="Which market/equity are you interested in?",  # Interface title
    description="Enter the name of the market or equity you're interested in."
)

# Launch the interface
iface.launch()


Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 42ms/step


full script

In [82]:
# from fastapi import FastAPI
# import tensorflow as tf
# from tensorflow.keras.layers import TextVectorization
# from newsapi import NewsApiClient
# from datetime import datetime, timedelta
# from config import getAPI
# import re
# from nltk.corpus import stopwords


# model = tf.keras.models.load_model('model-best') # load best model trained in the jupyter notebook

# app = FastAPI() # initialize FastAPI application

# print(model.summary())

# def preprocess(text):
#     text = text.lower()

#     # remove punctuations and numbers
#     text = re.sub('[^a-zA-Z]', ' ', text)

#     text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)  # when we remove apostrophe, is it replaced by a white space

#     # remove all single characters and replace it by a space
#     text = re.sub(r'\s+', ' ', text)

#     # remove stopwords
#     stop_words = stopwords.words('english')
#     pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')

#     text = pattern.sub('', text)
#     return text


# def processArticles(articles):
#     list = []
#     for article in articles['articles']:
#         # get title and description, defaulting to an empty string if they're not strings
#         title = article.get('title', '')
#         description = article.get('description', '')
#         # further make sure they are strings by replacing with '' if not string
#         if not isinstance(title, str):
#             title = ''
#         if not isinstance(description, str):
#             description = ''
#         title_and_description = preprocess(title + ' ' + description)
#         list.append(title_and_description)
#     return list


# def getArticles(input):
#     # Initialize the client with your API key
#     api_key = getAPI()
#     newsapi = NewsApiClient(api_key=api_key)

#     # Get today's date in the format YYYY-MM-DD
#     yesterday = (datetime.today() - timedelta(10)).strftime('%Y-%m-%d')
#     today = datetime.today().strftime('%Y-%m-%d')

#     # Fetch articles related to Tesla from today
#     articles = newsapi.get_everything(
#         q=input,
#         from_param=yesterday,
#         to=today,
#         language='en',
#         sort_by='relevancy'
#     )

#     return processArticles(articles)


# text_vectorization = TextVectorization(
#     max_tokens=68000,
#     output_sequence_length=int(78.0)
# )


# def run_model(inp):
#     # Get the list of articles or sentences
#     list = getArticles(inp)

#     if not list:
#         return "Empty! No recent news available here."

#     # Initialize a dictionary to keep track of sentiment counts
#     sentiment_counts = {"Negative": 0, "Neutral": 0, "Positive": 0}

#     # Predict sentiment for each input sentence
#     for input_sentence in list:
#         predictions = model.predict([input_sentence])

#         # Get the predicted class
#         predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

#         # Map the predicted class to the corresponding sentiment label
#         sentiment_labels = ["Negative", "Neutral", "Positive"]
#         predicted_sentiment = sentiment_labels[predicted_class]

#         # Increment the count for the predicted sentiment
#         sentiment_counts[predicted_sentiment] += 1

#     return sentiment_counts

In [10]:
import requests
url = ''
topic = "bitcoin"

In [11]:
params = {"topic": topic} 
response = requests.get(url, params=params)
response

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /analyze/?topic=bitcoin (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AB93B646D0>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))

In [12]:
import requests

url = 'http://127.0.0.1:8000/analyze/'
params = {"topic": topic} 
response = requests.get(url, params=params)

if response.status_code == 200:
    print(response.json())
else:
    print(f"Request failed with status code {response.status_code}")

{'articles': [{'title': 'Crypto Bro Charters Private SpaceX Mission to Earth’s Poles', 'description': "Fram2 could fly as early as this year, marking SpaceX's seventh private crew of astronauts.", 'url': 'https://gizmodo.com/crypto-bro-charters-private-spacex-mission-to-earths-poles-2000486329', 'sentiment': 'neutral'}, {'title': 'This Could Be the Beginning of the End of Trump’s Love for Crypto Cash', 'description': 'The Donald chose the wrong time to get into web3.', 'url': 'https://gizmodo.com/this-could-be-the-beginning-of-the-end-of-trumps-love-for-crypto-cash-2000483266', 'sentiment': 'positive'}, {'title': 'Bitcoin price crash: Why is the crypto market collapsing?', 'description': '', 'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_5f1ff72c-257b-4ef6-bc2a-c7c85f5106e9', 'sentiment': 'negative'}, {'title': 'What Happened in Crypto Today: Bullish Signals Are Finally Emerging!', 'description': 'From Bitcoin bouncing back above 56K to permanent holders qui